In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain.document_loaders import TextLoader

# Load document with LangChain's TextLoader
    
loader = TextLoader('./AI_executive_order_oct_2023.txt')
documents = loader.load()

# Chunk the document

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 696, which is longer than the specified 500
Created a chunk of size 572, which is longer than the specified 500
Created a chunk of size 1186, which is longer than the specified 500
Created a chunk of size 1289, which is longer than the specified 500
Created a chunk of size 988, which is longer than the specified 500
Created a chunk of size 1452, which is longer than the specified 500
Created a chunk of size 1030, which is longer than the specified 500
Created a chunk of size 887, which is longer than the specified 500
Created a chunk of size 1022, which is longer than the specified 500
Created a chunk of size 1012, which is longer than the specified 500
Created a chunk of size 532, which is longer than the specified 500
Created a chunk of size 533, which is longer than the specified 500
Created a chunk of size 604, which is longer than the specified 500
Created a chunk of size 514, which is longer than the specified 500
Created a chunk of size 1193, which is lon

In [3]:
# Create embeddings and vector store

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
    embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

embedded weaviate is already listening on port 6666


In [4]:
# Retriever

retriever = vectorstore.as_retriever()

# Augment

from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you do not know.
Use three setences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you do not know.\nUse three setences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n", template_format='f-string', validate_template=True), additional_kwargs={})]


In [5]:
# Generate

from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser ()
)

In [28]:
# Query[0]:

query = "How will the Secretary of Homeland Security create an AI Safety Board?"
rag_chain.invoke(query)

# Output: "The Secretary of Homeland Security will establish an Artificial Intelligence
# Safety and Security Board as an advisory committee. The board will include AI experts
# from the private sector, academia, and government. Its purpose is to provide advice,
# information, or recommendations for improving security, resilience, and incident response
# related to AI usage in critical infrastructure."

'The Secretary of Homeland Security will establish an Artificial Intelligence Safety and Security Board as an advisory committee. The board will include AI experts from the private sector, academia, and government. Its purpose is to provide advice, information, or recommendations for improving security, resilience, and incident response related to AI usage in critical infrastructure.'

In [29]:
# Query[1]:

query = "What does the AI Executive Order say about watermarking?"
rag_chain.invoke(query)

# Output: "The AI Executive Order defines watermarking as the act of
# embedding difficult-to-remove information into AI outputs for the
# purpose of verifying authenticity, identity, modifications, or conveyance."

'The AI Executive Order defines watermarking as the act of embedding difficult-to-remove information into AI outputs for the purpose of verifying authenticity, identity, modifications, or conveyance.'

In [30]:
# Query[2]:

query = "What is the theoretical maximum computing capacity cutoff for the technical conditions for AI models?"
rag_chain.invoke(query)

# Output: "The theoretical maximum computing capacity cutoff for the technical conditions for AI models is 1020
# integer or floating-point operations per second for training AI."
#
# **Note: 1020 means 10^20, and is correct based on the original text document which omits the '^' sign

'The theoretical maximum computing capacity cutoff for the technical conditions for AI models is 1020 integer or floating-point operations per second for training AI.'

In [7]:
# Query[2]:

query = "What is the easiest way to time travel?"
rag_chain.invoke(query)

# Output: "I do not know the answer."

'I do not know the answer.'